In [4]:
import numpy as np 
import pandas as pd 
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
warnings.simplefilter(action='ignore')
import gc
import dateutil.relativedelta

pd.set_option('display.width',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 200)

In [5]:
# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2    
#     for col in df.columns:
#         col_type = df[col].dtypes
#         if col_type in numerics:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)    
#     end_mem = df.memory_usage().sum() / 1024**2
#     if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
#     return df

In [6]:
historical_transactions = pd.read_csv('../input/historical_transactions.csv', parse_dates=['purchase_date'])
new_transactions = pd.read_csv('../input/new_merchant_transactions.csv', parse_dates=['purchase_date'])

In [7]:
np.percentile(historical_transactions['purchase_amount'], [1, 5, 50, 95, 99])

array([-0.74324133, -0.73819242, -0.68834948, -0.16861249,  1.22084097])

In [8]:
np.percentile(new_transactions['purchase_amount'], [1, 5, 50, 95, 99])

array([-0.74239984, -0.73638924, -0.67484064, -0.08574128,  1.4628174 ])

In [9]:
# historical_transactions['purchase_amount'] = historical_transactions['purchase_amount'].apply(lambda x: min(x, 1.5))

In [10]:
# new_transactions['purchase_amount'] = new_transactions['purchase_amount'].apply(lambda x: min(x, 1.5))

In [11]:
historical_transactions['category_2'] = historical_transactions['category_2'].fillna(-1, )
new_transactions['category_2'] = new_transactions['category_2'].fillna(-1, )

historical_transactions['category_3'] = historical_transactions['category_3'].map({'A':0, 'B':1, 'C':2})
new_transactions['category_3'] = new_transactions['category_3'].map({'A':0, 'B':1, 'C':2})

historical_transactions['category_3'] = historical_transactions['category_3'].fillna(-1, )
new_transactions['category_3'] = new_transactions['category_3'].fillna(-1, )

historical_transactions['merchant_id'] = historical_transactions['merchant_id'].fillna('-1', )
new_transactions['merchant_id'] = new_transactions['merchant_id'].fillna('-1', )

In [12]:
historical_transactions['category_3'].unique()

array([ 0.,  1.,  2., -1.])

In [13]:
historical_transactions['purchase_date'].max()

Timestamp('2018-02-28 23:59:51')

In [14]:
historical_transactions['purchase_date'].min()

Timestamp('2017-01-01 00:00:08')

In [15]:
new_transactions['purchase_date'].max()

Timestamp('2018-04-30 23:59:59')

In [16]:
new_transactions['purchase_date'].min()

Timestamp('2017-03-01 03:24:51')

In [19]:
%%time
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    
    return df

train = read_data('../input/train.csv')
test = read_data('../input/test.csv')

target = train['target']

gc.collect()

CPU times: user 453 ms, sys: 68 µs, total: 453 ms
Wall time: 452 ms


In [20]:
test[test['card_id']=='C_ID_c27b4f80f7']

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaT,C_ID_c27b4f80f7,5,2,1


In [21]:
# tmp = historical_transactions[historical_transactions['card_id']=='C_ID_c27b4f80f7']['purchase_date'].min() 
test['first_active_month'][test['card_id']=='C_ID_c27b4f80f7'] = pd.to_datetime(datetime.date(2017, 3, 1))

In [22]:
# historical_transactions = historical_transactions.head(200)
# new_transactions = new_transactions.head(200)
# train = train.head(200)
# test = test.head(200)

In [23]:
def sub_month_lag(x):
    return x['purchase_date_first'] - dateutil.relativedelta.relativedelta(months=x['month_lag_first'])

def a2p(a, p):
    return (a.dt.date - p.dt.date).dt.days

# def a2r(a, r):
#     return (a.dt.year-r.dt.year)*12 + (a.dt.month - r.dt.month)

In [24]:
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [25]:
agg_fun = {
    'authorized_flag': ['mean'], 
}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

In [26]:
# historical_transactions.count()

In [27]:
# historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]

In [28]:
# historical_transactions.count()

In [29]:
a_train = train[['card_id', 'first_active_month']]
a_test = test[['card_id', 'first_active_month']]

In [30]:
a_tmp = pd.concat([a_train,a_test]).drop_duplicates().reset_index(drop=True)

In [31]:
historical_transactions = pd.merge(historical_transactions, a_tmp, on='card_id', how='left')
new_transactions = pd.merge(new_transactions, a_tmp, on='card_id', how='left')

In [32]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month
0,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2016-06-01
1,1,C_ID_4e6213e9bc,88,0,0,0.0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,2016-06-01
2,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,2016-06-01
3,1,C_ID_4e6213e9bc,88,0,0,0.0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,2016-06-01
4,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,2016-06-01


In [33]:
historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'])
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])

In [34]:
def get_r(history):
    agg_func = {
        'month_lag': ['first', ],
        'purchase_date': ['first', ],
        }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    agg_history['reference_date'] = agg_history.apply(sub_month_lag, axis=1)
    agg_history['reference_date'] = agg_history['reference_date'].apply(lambda x: x + dateutil.relativedelta.relativedelta(day=1, months=+1, days=-1))
    agg_history['now_date'] = agg_history['reference_date'].apply(lambda x: x + dateutil.relativedelta.relativedelta(day=1, months=+5, days=-1))

    agg_history.drop(columns=['month_lag_first', 'purchase_date_first'], inplace=True)

    return agg_history

In [35]:
hist_r = get_r(historical_transactions)

In [36]:
hist_r.head()

,card_id,reference_date,now_date
0,C_ID_00007093c1,2018-02-28 19:09:42,2018-06-30 19:09:42
1,C_ID_0001238066,2018-02-28 12:14:16,2018-06-30 12:14:16
2,C_ID_0001506ef0,2018-02-28 08:25:45,2018-06-30 08:25:45
3,C_ID_0001793786,2017-10-31 15:13:07,2018-02-28 15:13:07
4,C_ID_000183fdda,2018-02-28 22:13:09,2018-06-30 22:13:09


In [37]:
historical_transactions = pd.merge(historical_transactions, hist_r, on='card_id', how='left')
new_transactions = pd.merge(new_transactions, hist_r, on='card_id', how='left')

In [38]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,reference_date,now_date
0,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07
1,1,C_ID_4e6213e9bc,88,0,0,0.0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07
2,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07
3,1,C_ID_4e6213e9bc,88,0,0,0.0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07
4,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07


In [39]:
for df in [historical_transactions, new_transactions]:
    df['a2p'] = a2p(df['first_active_month'], df['purchase_date'])
    
    df['p2r'] = a2p(df['purchase_date'], df['reference_date'])
    df['p2now'] = a2p(df['purchase_date'], df['now_date'])
    
#     df['p_and_m'] = df['purchase_amount'] * (df['month_lag'].abs()+1)
    df['p_vs_m'] = df['purchase_amount'] / (df['month_lag'].abs()+1)
    
    df["installments"].replace(-1, np.NaN, inplace=True)
    df["installments"].replace(999, np.NaN, inplace=True)
    
#     df['p_and_i'] = df['purchase_amount'] * (df['installments'].abs()+1)
    df['p_vs_i'] = df['purchase_amount'] / (df['installments'].abs()+1)

#     df['year'] = df['purchase_date'].dt.year
    df['quarter'] = df['purchase_date'].dt.quarter

    df['month'] = df['purchase_date'].dt.month
    
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour

#     df['is_quarter_start'] = (df.purchase_date.dt.is_quarter_start).astype(int)
#     df['is_quarter_end'] = (df.purchase_date.dt.is_quarter_end).astype(int)  
    df['is_month_start'] = (df.purchase_date.dt.is_month_start).astype(int)
#     df['is_month_end'] = (df.purchase_date.dt.is_month_end).astype(int)  

    df['weekend'] = (df.purchase_date.dt.weekday>=5).astype(int)  

In [40]:
historical_transactions['a2p'].plot.line()

In [41]:
new_transactions['a2p'].plot.line()

In [42]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,reference_date,now_date,a2p,p2r,p2now,p_vs_m,p_vs_i,quarter,month,weekofyear,dayofweek,day,hour,is_month_start,weekend
0,1,C_ID_4e6213e9bc,88,0,0.0,0.0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07,-389,-248,-370,-0.078148,-0.703331,2,6,25,6,25,15,0,1
1,1,C_ID_4e6213e9bc,88,0,0.0,0.0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07,-409,-228,-350,-0.091641,-0.733128,3,7,28,5,15,12,0,1
2,1,C_ID_4e6213e9bc,88,0,0.0,0.0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07,-434,-203,-325,-0.102912,-0.720386,3,8,32,2,9,22,0,0
3,1,C_ID_4e6213e9bc,88,0,0.0,0.0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07,-458,-179,-301,-0.122559,-0.735352,3,9,35,5,2,10,0,1
4,1,C_ID_4e6213e9bc,88,0,0.0,0.0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,2016-06-01,2018-02-28 15:33:07,2018-06-30 15:33:07,-282,-355,-477,-0.060239,-0.722865,1,3,10,4,10,1,0,0


In [43]:
def aggregate_transactions_hist(history_):
        
    agg_func = {
#         'is_quarter_start': ['sum', 'mean'],
#         'is_quarter_end': ['sum', 'mean'],
        'is_month_start': ['mean'],
#         'is_month_end': ['sum', 'mean'],
        'weekend': ['mean'],
        'category_1': ['mean'],
        #
        
        'category_2': ['nunique', ], #
        'category_3': ['nunique', ], #
        'state_id': ['nunique', ],
        'city_id': ['nunique', ],
        'subsector_id': ['nunique', ],
        'merchant_category_id': ['nunique', ],
        'merchant_id': ['nunique', ],
#         'year': ['nunique', ],
        'quarter': ['nunique', ],
        'month': ['nunique', ], 
        'weekofyear': ['nunique', ],
        'dayofweek': ['nunique', ],
        'day': ['nunique', ],
        'hour': ['nunique', ],
        
        #
        'a2p': ['mean', 'median', 'max', 'min', 'std'], 
        
        'p2r': ['mean', 'median', 'max', 'min', 'std'], 
        'p2now': ['mean', 'median', 'max', 'min', 'std'],  ################
        
        'month_lag': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_amount': ['sum', 'mean', 'median', 'max', 'min', 'std'], 
        'installments': ['sum', 'mean', 'median', 'max', 'min', 'std'], 
#         'p_and_m': ['mean', 'median', 'max', 'min', 'std'], 
        'p_vs_m': ['mean', 'median', 'max', 'min', 'std'],
#         'p_and_i': ['mean', 'median', 'max', 'min', 'std'], 
        'p_vs_i': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_date': ['max', 'min'],
        'first_active_month': ['first'],
        'reference_date': ['first'],
        'now_date': ['first'],
        }
        
    history = history_
    
    for col in ['category_2', 'category_3', 
                'state_id', 'city_id', 'subsector_id', 'merchant_category_id', 'merchant_id', 
#                 'year', 
                'quarter', 
                'month', 'weekofyear', 
                'dayofweek',
                'day',
                'hour'
               ]:
        
#         freq_encode = history[col].value_counts(normalize=True)
#         history[col+'_freq'] = history[col].map(freq_encode)
        
        history[col+'_p_mean'] = history.groupby([col])['purchase_amount'].transform('mean')  # mean encode improve 0.697 to 0.694
#         history[col+'_i_mean'] = history.groupby([col])['installments'].transform('mean')

#         agg_func[col+'_freq'] = ['mean']

#         agg_func[col+'_p_mean'] = ['mean']
        agg_func[col+'_p_mean'] = ['mean', 'median', 'max', 'min', 'std']
#         agg_func[col+'_i_mean'] = ['mean', 'max', 'min', 'std']

    agg_history = history.groupby(['card_id']).agg(agg_func)
    
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    agg_history['first_year'] = agg_history['first_active_month_first'].dt.year
    agg_history['first_quarter'] = agg_history['first_active_month_first'].dt.quarter
    agg_history['first_month'] = agg_history['first_active_month_first'].dt.month
    
    agg_history['re_year'] = agg_history['reference_date_first'].dt.year
    agg_history['re_quarter'] = agg_history['reference_date_first'].dt.quarter
    agg_history['re_month'] = agg_history['reference_date_first'].dt.month
    
    agg_history['now_year'] = agg_history['now_date_first'].dt.year
    agg_history['now_quarter'] = agg_history['now_date_first'].dt.quarter
    agg_history['now_month'] = agg_history['now_date_first'].dt.month
    
    agg_history['a2r'] = a2p(agg_history['first_active_month_first'], agg_history['reference_date_first'])
    agg_history['r2now'] = a2p(agg_history['reference_date_first'], agg_history['now_date_first'])
    agg_history['a2now'] = a2p(agg_history['first_active_month_first'], agg_history['now_date_first']) #############

    agg_history.drop(columns=['first_active_month_first', 'reference_date_first', 'now_date_first'], inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

gc.collect()

33

In [44]:
%%time
history = aggregate_transactions_hist(historical_transactions)
    
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]

history['hist_p2p'] = (history['hist_purchase_date_max'] - history['hist_purchase_date_min']).dt.days
history['hist_sleep'] = history['hist_p2p'] - history['hist_day_nunique']

###
history['hist_p2p_vs_count'] = history['hist_p2p']/history['hist_transactions_count']
history['hist_sleep_vs_count'] = history['hist_sleep']/history['hist_transactions_count']
###
history['hist_count_vs_p2p'] = history['hist_transactions_count']/(history['hist_p2p'].abs()+1)
history['hist_sleep_vs_p2p'] = history['hist_sleep']/(history['hist_p2p'].abs()+1)
history['hist_p_vs_p2p'] = history['hist_purchase_amount_sum']/(history['hist_p2p'].abs()+1)

history['hist_i_vs_p2p'] = history['hist_installments_sum']/(history['hist_p2p'].abs()+1)

# history[:5]
gc.collect()

CPU times: user 5min 36s, sys: 28.5 s, total: 6min 5s
Wall time: 6min 4s


In [45]:
history.head()

,card_id,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_p2now_mean,hist_p2now_median,hist_p2now_max,hist_p2now_min,hist_p2now_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_first_year,hist_first_quarter,hist_first_month,hist_re_year,hist_re_quarter,hist_re_month,hist_now_year,hist_now_quarter,hist_now_month,hist_a2r,hist_r2now,hist_a2now,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,hist_i_vs_p2p
0,C_ID_00007093c1,149,0.067114,0.167785,0.187919,3,2,3,4,13,18,29,4,12,39,7,28,18,-197.838926,-185.0,-13,-391,104.216256,-194.161074,-207.0,-1,-379,104.216256,-316.161074,-329.0,-123,-501,104.216256,-5.852349,-6.0,0,-12,3.453114,-76.845041,-0.515739,-0.587627,1.507069,-0.728876,0.298141,192.0,1.288591,1.0,6.0,1.0,0.764900,-0.128244,-0.085235,0.215296,-0.721363,0.149488,-0.251965,-0.290808,0.376767,-0.364438,0.128764,2018-02-27 05:14:57,2017-02-14 14:00:43,0.077946,0.180375,0.180375,-0.354523,0.210037,-0.322315,-0.404556,0.106023,-0.404556,0.188321,0.166921,0.288618,0.288618,-0.354523,0.251892,0.289764,0.512332,0.512332,-0.359418,0.344451,-0.162991,-0.245685,2.180532,-0.678829,0.618114,-0.124108,-0.278704,7.143294,-0.698283,1.138669,-0.325462,-0

In [46]:
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')
del history; gc.collect()

14

In [47]:
def aggregate_transactions_new(history_):
        
    agg_func = {
#         'is_quarter_start': ['sum', 'mean'],
#         'is_quarter_end': ['sum', 'mean'],
        'is_month_start': ['mean'],
#         'is_month_end': ['sum', 'mean'],
        'weekend': ['mean'],
        'category_1': ['mean'],
        #
        
        'category_2': ['nunique', ], #
        'category_3': ['nunique', ], #
        'state_id': ['nunique', ],
        'city_id': ['nunique', ],
        'subsector_id': ['nunique', ],
        'merchant_category_id': ['nunique', ],
        'merchant_id': ['nunique', ],
#         'year': ['nunique', ],
        'quarter': ['nunique', ],
        'month': ['nunique', ], 
        'weekofyear': ['nunique', ],
        'dayofweek': ['nunique', ],
        'day': ['nunique', ],
        'hour': ['nunique', ],
        
        #
        'a2p': ['mean', 'median', 'max', 'min', 'std'],  ################
        
        'p2r': ['mean', 'median', 'max', 'min', 'std'], 
        'p2now': ['mean', 'median', 'max', 'min', 'std'], 
        
        'month_lag': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_amount': ['sum', 'mean', 'median', 'max', 'min', 'std'], # 
        'installments': ['sum', 'mean', 'median', 'max', 'min', 'std'], # 
#         'p_and_m': ['mean', 'median', 'max', 'min', 'std'], # 
        'p_vs_m': ['mean', 'median', 'max', 'min', 'std'], # 
#         'p_and_i': ['mean', 'median', 'max', 'min', 'std'], # 
        'p_vs_i': ['mean', 'median', 'max', 'min', 'std'], # 
        'purchase_date': ['max', 'min'],
        }
        
    history = history_
    
    for col in ['category_2', 'category_3', 
                'state_id', 'city_id', 'subsector_id', 'merchant_category_id', 'merchant_id', 
#                 'year', 
                'quarter', 
                'month', 'weekofyear', 
                'dayofweek',
                'day',
                'hour'
               ]:
        
#         freq_encode = history[col].value_counts(normalize=True)
#         history[col+'_freq'] = history[col].map(freq_encode)
        
        history[col+'_p_mean'] = history.groupby([col])['purchase_amount'].transform('mean')  # mean encode import 0.697 to 0.694
#         history[col+'_i_mean'] = history.groupby([col])['installments'].transform('mean')

#         agg_func[col+'_freq'] = ['mean']

#         agg_func[col+'_p_mean'] = ['mean']
        agg_func[col+'_p_mean'] = ['mean', 'median', 'max', 'min', 'std']
#         agg_func[col+'_i_mean'] = ['mean', 'max', 'min', 'std']

    agg_history = history.groupby(['card_id']).agg(agg_func)
    
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
        
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
        
    return agg_history

gc.collect()

0

In [48]:
%%time
new = aggregate_transactions_new(new_transactions)
    
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
    
new['new_p2p'] = (new['new_purchase_date_max'] - new['new_purchase_date_min']).dt.days
new['new_sleep'] = new['new_p2p'] - new['new_day_nunique']
###
new['new_p2p_vs_count'] = new['new_p2p']/new['new_transactions_count']
new['new_sleep_vs_count'] = new['new_sleep']/new['new_transactions_count']
###
new['new_count_vs_p2p'] = new['new_transactions_count']/(new['new_p2p'].abs()+1)
new['new_sleep_vs_p2p'] = new['new_sleep']/(new['new_p2p'].abs()+1)
new['new_p_vs_p2p'] = new['new_purchase_amount_sum']/(new['new_p2p'].abs()+1)

new['new_i_vs_p2p'] = new['new_installments_sum']/(new['new_p2p'].abs()+1)

# new[:5]
gc.collect()

CPU times: user 29.7 s, sys: 499 ms, total: 30.2 s
Wall time: 30.2 s


In [49]:
new.head()

,card_id,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_p2now_mean,new_p2now_median,new_p2now_max,new_p2now_min,new_p2now_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_median,new_installments_max,new_installments_min,new_installments_std,new_p_vs_m_mean,new_p_vs_m_median,new_p_vs_m_max,new_p_vs_m_min,new_p_vs_m_std,new_p_vs_i_mean,new_p_vs_i_median,new_p_vs_i_max,new_p_vs_i_min,new_p_vs_i_std,new_purchase_date_max,new_purchase_date_min,new_category_2_p_mean_mean,new_category_2_p_mean_median,new_category_2_p_mean_max,new_category_2_p_mean_min,new_category_2_p_mean_std,new_category_3_p_mean_mean,new_category_3_p_mean_median,new_category_3_p_mean_max,new_category_3_p_mean_min,new_category_3_p_mean_std,new_state_id_p_mean_mean,new_state_id_p_mean_median,new_state_id_p_mean_max,new_state_id_p_mean_min,new_state_id_p_mean_std,new_city_id_p_mean_mean,new_city_id_p_mean_median,new_city_id_p_mean_max,new_city_id_p_mean_min,new_city_id_p_mean_std,new_subsector_id_p_mean_mean,new_subsector_id_p_mean_median,new_subsector_id_p_mean_max,new_subsector_id_p_mean_min,new_subsector_id_p_mean_std,new_merchant_category_id_p_mean_mean,new_merchant_category_id_p_mean_median,new_merchant_category_id_p_mean_max,new_merchant_category_id_p_mean_min,new_merchant_category_id_p_mean_std,new_merchant_id_p_mean_mean,new_merchant_id_p_mean_median,new_merchant_id_p_mean_max,new_merchant_id_p_mean_min,new_merchant_id_p_mean_std,new_quarter_p_mean_mean,new_quarter_p_mean_median,new_quarter_p_mean_max,new_quarter_p_mean_min,new_quarter_p_mean_std,new_month_p_mean_mean,new_month_p_mean_median,new_month_p_mean_max,new_month_p_mean_min,new_month_p_mean_std,new_weekofyear_p_mean_mean,new_weekofyear_p_mean_median,new_weekofyear_p_mean_max,new_weekofyear_p_mean_min,new_weekofyear_p_mean_std,new_dayofweek_p_mean_mean,new_dayofweek_p_mean_median,new_dayofweek_p_mean_max,new_dayofweek_p_mean_min,new_dayofweek_p_mean_std,new_day_p_mean_mean,new_day_p_mean_median,new_day_p_mean_max,new_day_p_mean_min,new_day_p_mean_std,new_hour_p_mean_mean,new_hour_p_mean_median,new_hour_p_mean_max,new_hour_p_mean_min,new_hour_p_mean_std,new_p2p,new_sleep,new_p2p_vs_count,new_sleep_vs_count,new_count_vs_p2p,new_sleep_vs_p2p,new_p_vs_p2p,new_i_vs_p2p
0,C_ID_00007093c1,2,0.000000,0.000000,0.000000,2,1,2,2,2,2,2,1,1,2,2,2,2,-429.000000,-429.0,-426,-432,4.242641,37.000000,37.0,40,34,4.242641,-85.000000,-85.0,-82,-88,4.242641,2.000000,2.0,2,2,0.000000,-1.328524,-0.664262,-0.664262,-0.656749,-0.671775,0.010625,2.0,1.00,1.0,1.0,1.0,0.000000,-0.221421,-0.221421,-0.218916,-0.223925,0.003542,-0.332131,-0.332131,-0.328374,-0.335888,0.005313,2018-04-09 16:23:59,2018-04-03 11:13:35,-0.560047,-0.560047,-0.550852,-0.569242,0.013004,-0.606486,-0.606486,-0.606486,-0.606486,0.000000,-0.560864,-0.560864,-0.557145,-0.564584,0.005260,-0.555300,-0.555300,-0.546606,-0.563993,0.012294,-0.457642,-0.457642,-0.365697,-0.549586,0.130029,-0.446722,-0.446722,-0.379453,-0.513992,0.095133,-0.612127,-0.612127,-0.552479,-0.671775,0.084355,-0.547688,-0.547688,-0.547688,-0.547688,0.000000,-0.545173,-0.545173,-0.545173,-0.545173,0.000000,-0.546550,-0.546550,-0.546106,-0.546995,0.000629,-0.533363,-0.533363,-0.532678,-0.534049,0.000969,-0.548294,-0.548294,-0.543283,-0.553305,0.007087,-0.530753,-0.530753,-0.527109,-0.534396,0.00

In [50]:
train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')
del new; gc.collect()

28

In [51]:
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [52]:
for df in [train, test]:
    ###
    df['c_p2p_diff'] = df['hist_p2p_vs_count'] - df['new_p2p_vs_count']
    df['c_sleep_diff'] = df['hist_sleep_vs_count'] - df['new_sleep_vs_count']
    df['c_p_diff'] = df['hist_purchase_amount_mean'] - df['new_purchase_amount_mean']
    df['c_i_diff'] = df['hist_installments_mean'] - df['new_installments_mean']
    ###
    df['p2p_count_diff'] = df['hist_count_vs_p2p'] - df['new_count_vs_p2p']
    df['p2p_sleep_diff'] = df['hist_sleep_vs_p2p'] - df['new_sleep_vs_p2p']
    df['p2p_p_diff'] = df['hist_p_vs_p2p'] - df['new_p_vs_p2p']
    df['p2p_i_diff'] = df['hist_i_vs_p2p'] - df['new_i_vs_p2p']
    ###
    df['c_p2p_diff_vs'] = df['c_p2p_diff'] / df['hist_p2p_vs_count']
    df['c_sleep_diff_vs'] = df['c_sleep_diff'] / df['hist_sleep_vs_count']
    df['c_p_diff_vs']  = df['c_p_diff'] / df['hist_purchase_amount_mean']
    df['c_i_diff_vs'] = df['c_i_diff'] / df['hist_installments_mean']
    ###
    df['p2p_count_diff_vs'] = df['p2p_count_diff'] / df['hist_count_vs_p2p']
    df['p2p_sleep_diff_vs'] = df['p2p_sleep_diff'] / df['hist_sleep_vs_p2p']
    df['p2p_p_diff_vs']  = df['p2p_p_diff'] / df['hist_p_vs_p2p']
    df['p2p_i_diff_vs'] = df['p2p_i_diff'] / df['hist_i_vs_p2p']
    ###
#     df['count_sum'] = df['hist_transactions_count'] + df['new_transactions_count']
#     df['p_sum'] = df['hist_purchase_amount_sum'] + df['new_purchase_amount_sum']
#     df['i_sum'] = df['hist_installments_sum'] + df['new_installments_sum']
#     df['p_sum_vs_count'] = df['p_sum'] / df['count_sum'] # mean
#     df['i_sum_vs_count'] = df['i_sum'] / df['count_sum'] # mean
    ###
#     df['gap'] = a2p(df['hist_purchase_date_max'], df['new_purchase_date_min'])
    ###
#     df['time'] = a2p(df['hist_purchase_date_min'], df['new_purchase_date_max'])
#     df['sleep_sum'] = df['hist_sleep'] + df['new_sleep']

#     df['time_vs_count'] = df['time'] / df['count_sum']
#     df['sleep_sum_vs_count'] = df['sleep_sum'] / df['count_sum']
    
#     df['count_sum_vs_time'] = df['count_sum'] / (df['time'].abs() + 1)
#     df['sleep_sum_vs_time'] = df['sleep_sum'] / (df['time'].abs() + 1)
#     df['p_sum_vs_time'] = df['p_sum'] / (df['time'].abs() + 1)
#     df['i_sum_vs_time'] = df['i_sum'] / (df['time'].abs() + 1)
    ###
    
    ###
    for f in ['hist_purchase_date_max','hist_purchase_date_min', 
              'new_purchase_date_max', 'new_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9

In [53]:
train['hist_p2p'].plot.hist(bins=50)

In [54]:
train['new_p2p'].plot.hist(bins=50)

In [55]:
train['hist_sleep'].plot.hist(bins=50)

In [56]:
train['new_sleep'].plot.hist(bins=50)

In [57]:
train['c_p2p_diff'].plot.hist(bins=50)

In [58]:
# train['c_sleep_diff'].plot.hist(bins=50)

In [59]:
train['c_p_diff'].apply(lambda x: min(x, 10)).plot.hist(bins=50)

In [60]:
train['c_i_diff'].plot.hist(bins=50)

In [61]:
# train['p2p_sleep_diff'].plot.hist(bins=50)

In [62]:
# train['p2p_count_diff'].plot.hist(bins=50)

In [63]:
train['p2p_p_diff'].apply(lambda x: min(x, 10)).plot.hist(bins=50)

In [64]:
train['p2p_i_diff'].plot.hist(bins=50)

In [65]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,authorized_flag_mean,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_p2now_mean,hist_p2now_median,hist_p2now_max,hist_p2now_min,hist_p2now_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_first_year,hist_first_quarter,hist_first_month,hist_re_year,hist_re_quarter,hist_re_month,hist_now_year,hist_now_quarter,hist_now_month,hist_a2r,hist_r2now,hist_a2now,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,hist_i_vs_p2p,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_p2now_mean,new_p2now_median,new_p2now_max,new_p2now_min,new_p2now_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase

In [66]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,authorized_flag_mean,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_p2now_mean,hist_p2now_median,hist_p2now_max,hist_p2now_min,hist_p2now_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_first_year,hist_first_quarter,hist_first_month,hist_re_year,hist_re_quarter,hist_re_month,hist_now_year,hist_now_quarter,hist_now_month,hist_a2r,hist_r2now,hist_a2now,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,hist_i_vs_p2p,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_p2now_mean,new_p2now_median,new_p2now_max,new_p2now_min,new_p2now_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount

In [67]:
train['authorized_flag_mean'].mean()

0.8933534742623435

In [68]:
test['authorized_flag_mean'].mean()

0.8930119505693119

In [69]:
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
gc.collect()

Train Shape: (201917, 304)
Test Shape: (123623, 302)


90

In [70]:
def summary(df):
    stats = []
    for col in df.columns:
        stats.append((col, df[col].nunique(), 
                      df[col].isnull().sum() * 100 / df.shape[0], 
                      df[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                      df[col].dtype))
    
    return pd.DataFrame(stats, columns=['feature', 'unique', 'missing', 'mode', 'type'])

In [71]:
train_summary = summary(train)
test_summary = summary(test)

In [72]:
tmp_1 = train_summary.sort_values('mode', ascending=False)
tmp_2 = test_summary.sort_values('mode', ascending=False)

In [73]:
tmp_1

,feature,unique,missing,mode,type
287,outliers,2,0.000000,98.906977,int64
121,hist_dayofweek_p_mean_min,7,0.000000,93.436907,float64
139,hist_now_year,2,0.000000,92.246319,int64
106,hist_quarter_p_mean_min,4,0.000000,91.651025,float64
41,hist_month_lag_max,12,0.000000,89.991927,int64
120,hist_dayofweek_p_mean_max,7,0.000000,86.648970,float64
140,hist_now_quarter,4,0.000000,83.931516,int64
137,hist_re_quarter,4,0.000000,79.116667,int64
131,hist_hour_p_mean_min,23,0.000000,78.895289,float64
136,hist_re_year,2,0.000000,78.289594,int64


In [74]:
tmp_2

,feature,unique,missing,mode,type
120,hist_dayofweek_p_mean_min,6,0.000000,93.430834,float64
138,hist_now_year,2,0.000000,92.364689,int64
105,hist_quarter_p_mean_min,4,0.000000,91.699765,float64
40,hist_month_lag_max,12,0.000000,90.163643,int64
119,hist_dayofweek_p_mean_max,7,0.000000,86.591492,float64
139,hist_now_quarter,4,0.000000,84.186600,int64
136,hist_re_quarter,4,0.000000,79.254669,int64
130,hist_hour_p_mean_min,22,0.000000,78.753145,float64
135,hist_re_year,2,0.000000,78.478115,int64
153,new_is_month_start_mean,209,11.007660,76.184852,float64


In [75]:
train.to_csv("./data/pre_train.csv")

In [76]:
test.to_csv("./data/pre_test.csv")

In [77]:
tmp = train['target']

In [78]:
tmp.plot.hist(bins=50)

In [79]:
(tmp*np.log10(2)).plot.hist(bins=50)

In [80]:
np.log2((np.exp2(tmp) - 0.0000000001) + 1).plot.hist(bins=50)